# Code 19

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=1500).fit(X, y)
X_sel = selector.transform(X)
print(X_sel.shape)

# Code 20

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

def positive_indicatorsRLR(X, y, target, vectorizer, selection_threshold=0.3, num_iters=100):
    n_instances, n_feats = X.shape
    
    pos_scores = [] # all coefficient > 0
    neg_scores = [] # all coefficient < 0
    # choices for lambda weight
    penalities = [10,5,2,1,0.5,0.1,0.05,0.01,0.005,0.001,0.0001, 0.00001]

    # select repeated subsamples
    for iteration in range(num_iters):
        # initialize a model with randomly-weighted L1 penalty
        clf = LogisticRegression(penalty='l1', C=penalities[np.random.randint(len(penalities))])

    # choose a random subset of indices of the data with replacement
        selection = np.random.choice(n_instances, size=int(n_instances * 0.75))
        try:
            clf.fit(X[selection], y[selection])
        except ValueError:
            continue
    
    # record which coefficients got a positive or negative score
        pos_scores.append(clf.coef_ > 0)
        neg_scores.append(clf.coef_ < 0)

    # normalize the counts
    pos_scores = (np.array(pos_scores).sum(axis=0)/num_iters).reshape(-1)
    neg_scores = (np.array(neg_scores).sum(axis=0)/num_iters).reshape(-1)

    # find the features corresponding to the non-zero coefficients
    features = vectorizer.get_feature_names()
    pos_positions = [i for i, v in enumerate(pos_scores >= selection_threshold) if v]
    neg_positions = [i for i, v in enumerate(neg_scores >= selection_threshold) if v]

    pos = [(features[i], pos_scores[i]) for i in pos_positions]
    neg = [(features[i], neg_scores[i]) for i in neg_positions]

    posdf = pd.DataFrame(pos, columns='term score'.split()).sort_values('score', ascending=False)
    negdf = pd.DataFrame(neg, columns='term score'.split()).sort_values('score', ascending=False)

    return posdf, negdf